# Generative Artificial Intelligence
---

Get started with LLMs

---

## 📑 Contents

1. OpenAI API - GPT
2. Anthropic API - Claude
3. Google API - Gemini
4. HuggingFace API
5. OpenAI Embeddings
6. HuggingFace Embeddings
7. Mini Project

## 1. String Output Parsers

In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

prompt1 = template1.invoke({'topic':'black hole'})

result = model.invoke(prompt1)

prompt2 = template2.invoke({'text':result.content})

result1 = model.invoke(prompt2)

print(result1.content)


Black holes are dense regions in spacetime where gravity is so strong that nothing, not even light, can escape. They form from the collapse of massive stars, creating singularities and event horizons. Black holes exhibit properties such as mass and spin, impacting surrounding environments, including warping spacetime and driving the formation of accretion disks. Through gravitational lensing, accreting materials, and X-ray emissions, astronomers have observed these fascinating objects. Research into black holes provides insights into the development of galaxies, the evolution of the universe, and even the fabric of spacetime itself. 



## 1.1 String Output parsers using Chain

In [2]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | model | parser | template2 | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)
print(result1.content)


Here is a 5-line summary of the text provided:

Black holes are regions of spacetime where gravity is so strong that even light cannot escape. Formed from the collapse of massive stars, they have an event horizon where escape velocity exceeds the speed of light, and a singularity at their core. Their presence can be detected through various phenomena, like accretion disks, gravitational lensing, and gravitational waves.  They play a crucial role in galaxy evolution, as well as our quest to understand the fundamental laws of physics through studying their properties. Ongoing research seeks to answer fundamental questions about their nature and the composition of their singularity. 

Black holes are dense regions in spacetime where gravity is so strong that nothing, not even light, can escape. They form from the collapse of massive stars, creating singularities and event horizons. Black holes exhibit properties such as mass and spin, impacting surrounding environments, including warping 

## 2. JSON Output Parser

In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)


[{'fact': 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'detail': 'This is due to an extremely dense concentration of mass packed into a microscopic point called a singularity.'}, {'fact': 'Black holes are formed when large stars run out of fuel and collapse under their own gravity.', 'detail': 'The collapse can create a black hole whose mass is equivalent to the mass of many suns.'}, {'fact': 'Black holes come in different sizes. There are stellar-mass black holes formed by extremely massive stars, supermassive black holes found at the center of galaxies, and intermediate-mass black holes which are still being explored.', 'detail': 'The size varies from the stellar mass to supermassive depending on their formation process.'}, {'fact': 'Black holes can pull in surrounding matter, eventually warping space and time around them', 'detail': 'This warping creates a region known as the event horizon. Anything crossing this bounda

## 3. Structured Output Parser

In [4]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'fact_1': 'A black hole is an area of spacetime where gravity is so strong that nothing, not even light, can escape.', 'fact_2': 'Black holes are formed when massive stars collapse at the end of their lives.', 'fact_3': "We can't see a black hole directly because it doesn't emit any light. However, we can detect them by observing their effects on nearby objects, such as stars orbiting a central point that disappears from view."}


## 4. Pydantic Output Parser

In [5]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

final_result = chain.invoke({'place':'Korean'})

print(final_result)

name='Seunghee Park' age=25 city='Seoul'
